In [1]:
import pandas as pd
path="./data/javatpoint.json"
df = pd.read_json(path)

In [8]:
len(df)

565

In [2]:
from dotenv import load_dotenv
import os
load_dotenv()
from pinecone import Pinecone 
p=Pinecone(api_key=os.getenv("PINECONE_KEY"),environment=os.getenv("PINECONE_ENVIRONMENT"))
from sentence_transformers import SentenceTransformer
# model=SentenceTransformer("all-mpnet-base-v2") # dim 768

In [3]:
model2=SentenceTransformer("msmarco-distilbert-base-v3")

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

C:\Users\Sushanta Das\AppData\Roaming\Python\Python310\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Sushanta Das\.cache\huggingface\hub\models--sentence-transformers--msmarco-distilbert-base-v3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/499 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
embeding=model2.encode("hellow Tamal")
len(embeding)

768

In [51]:
index=p.Index('search')
meta_data=df.copy()
meta_data=meta_data.drop(columns=['link'])
meta_data.head()

title  \
0  Learn Python Tutorial | Python Programming Lan...   
1                 Python SciPy Tutorial - javatpoint   
2                Python Design Patterns - javatpoint   
3                Python CGI Programming - javatpoint   
4  Python Pandas | Python Pandas Tutorial - javat...   

                                                h1  \
0  [Python Tutorial | Python Programming Language]   
1                          [Python SciPy Tutorial]   
2                          [Python Design Pattern]   
3                         [Python CGI Programming]   
4                         [Python Pandas Tutorial]   

                                                  h2  \
0  [Python Tutorial, Python OOPs, Python MySQL, P...   
1  [Python SciPy Tutorial, What is SciPy, History...   
2  [Design Pattern, What is Design Pattern?, Conf...   
3  [Python CGI Programming, What is CGI?, Web Bro...   
4  [Pandas Tutorial, Pandas Series, Pandas DataFr...   

                                              strong  
0  [Python, Java, C++., Guido van Rossum, Python ...  
1  [Scipy, Sigh pi, SciPy, integration, gradient ...  
2  [For example -, Pattern Name -, Intent/ Motive...  
3  ["Common Gateway Interface", <form>, <isindex>...  
4  [Representation of Data:, Clear code:, Data:, ...

In [145]:
query_list=[]

for i,row in df.iterrows():
    encoding= model2.encode(row['title']).tolist()
    query_list.append(
        (
            str(i),
            encoding,
            {
                "link":str(row["link"]),
                "title":str(row["title"])
            }
        )
    )
    # if len(query_list)==50 or i==len(df)-1:
    #     index.upsert(vectors=query_list)
    #     query_list=[]

In [147]:
query_list[5][1]
# index.upsert(vectors=query_list[:20])

[-0.48594361543655396,
 0.1944010704755783,
 -0.7913736701011658,
 -0.29425886273384094,
 0.5593632459640503,
 0.03188508749008179,
 0.16363386809825897,
 0.34550905227661133,
 0.4094461500644684,
 0.7255523800849915,
 -0.028669029474258423,
 1.0047134160995483,
 0.07952596247196198,
 -0.9472774863243103,
 -0.18176600337028503,
 -0.5154730081558228,
 0.03313944488763809,
 -0.6323642730712891,
 0.5719645619392395,
 0.15281042456626892,
 0.1480332911014557,
 0.26008012890815735,
 -1.2535780668258667,
 0.23378506302833557,
 -0.37501952052116394,
 0.5352221727371216,
 0.18882142007350922,
 -0.25839993357658386,
 -1.0531933307647705,
 -0.13631834089756012,
 -0.4973152279853821,
 0.20696769654750824,
 0.29794856905937195,
 -0.3389328718185425,
 0.18261833488941193,
 -0.18875345587730408,
 -0.43800869584083557,
 0.10963613539934158,
 0.18854190409183502,
 0.27371224761009216,
 0.5983638763427734,
 0.13386477530002594,
 0.7270323038101196,
 0.10536956042051315,
 -0.13318778574466705,
 0.264188

In [5]:
from pinecone import Pinecone 
pc=Pinecone(api_key=os.getenv("PINECONE_KEY"),environment=os.getenv("PINECONE_ENVIRONMENT"))
index=pc.Index('search')

In [148]:
index.upsert(
    vectors=query_list,
    namespace= "ns1"
)
# [query_list]
# for i,row in enumerate(query_list):
#     index.upsert
#index.upsert(
#     vectors=[
#         {
#             "id": query_list[0][0], 
#             "values": query_list[0][1], 
#             "metadata": query_list[0][2]
#         },
#         {
#             "id": query_list[1][0], 
#             "values": query_list[1][1], 
#             "metadata": query_list[1][2]
#         },
#         {
#             "id": query_list[2][0], 
#             "values": query_list[2][1], 
#             "metadata": query_list[2][2]
#         }
#     ],
#     namespace= "ns3"
# )
# query_list[2][0]

{'upserted_count': 565}

In [110]:
index.query(
    namespace="ns0",
    vector=query_list[0][1],
    top_k=1,
    include_values=True,
    include_metadata=True,
    # filter={"genre": {"$eq": "action"}}
)

{'matches': [{'id': '0',
              'metadata': {'link': 'https://www.javatpoint.com/python-tutorial',
                           'title': 'Learn Python Tutorial | Python '
                                    'Programming Language - javatpoint'},
              'score': 1.00080717,
              'values': [-0.00758785242,
                         -0.00694684591,
                         -0.0111509673,
                         0.0143307932,
                         -0.0257320274,
                         0.0265319273,
                         -0.00286593451,
                         0.0231264196,
                         0.0390201621,
                         -0.0298129078,
                         0.0166312642,
                         0.0334960036,
                         -0.0319201052,
                         0.0243059527,
                         0.000456694252,
                         -0.030150827,
                         -0.038491834,
                         0.0189297404,
 

In [12]:
def search_pinecone(model,query,table,namespace,res):
    pc=Pinecone(api_key=os.getenv("PINECONE_KEY"),environment=os.getenv("PINECONE_ENVIRONMENT"))
    index=pc.Index(table)
    
    result=index.query(
        namespace=namespace,
        vector=model.encode(query).tolist(),
        top_k=res,
        include_values=False,
        include_metadata=True,
        # filter={"genre": {"$eq": "action"}}
    )
    return result
results=search_pinecone(model2,"object",'search','ns1',5)
results['matches']

[{'id': '86',
  'metadata': {'link': 'https://www.javatpoint.com/python-object-function',
               'title': 'object() in Python | Python object() Function with '
                        'Examples - Javatpoint'},
  'score': 0.404614627,
  'values': []},
 {'id': '371',
  'metadata': {'link': 'https://www.javatpoint.com/python-objects-classes',
               'title': 'Classes and Objects in Python - javatpoint'},
  'score': 0.313920498,
  'values': []},
 {'id': '36',
  'metadata': {'link': 'https://www.javatpoint.com/python-iter-function',
               'title': 'iter() in Python | Python iter() Function with '
                        'Examples - Javatpoint'},
  'score': 0.225537539,
  'values': []},
 {'id': '383',
  'metadata': {'link': 'https://www.javatpoint.com/python-opencv-object-detection',
               'title': 'Python OpenCV object detection - Javatpoint'},
  'score': 0.203307763,
  'values': []},
 {'id': '107',
  'metadata': {'link': 'https://www.javatpoint.com/python-

In [1]:
import requests
from bs4 import BeautifulSoup

def get_webpage_content(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        text_content = ' '.join([p.get_text() for p in soup.find_all('p')])
        return text_content
    except Exception as e:
        print(f"Error fetching webpage content: {e}")
        return None

get_webpage_content("https://www.geeksforgeeks.org/simple-chat-room-using-python/?ref=ml_lbp")

'This article demonstrates – How to set up a simple Chat Room server and allow multiple clients to connect to it using a client-side script. The code uses the concept of sockets and threading.\xa0\xa0 Sockets can be thought of as endpoints in a communication channel that is bi-directional and establishes communication between a server and one or more clients. Here, we set up a socket on each end and allow a client to interact with other clients via the server. The socket on the server side associates itself with some hardware port on the server-side. Any client that has a socket associated with the same port can communicate with the server socket.\xa0\xa0 A thread is a sub-process that runs a set of commands individually of any other thread. So, every time a user connects to the server, a separate thread is created for that user, and communication from the server to the client takes place along individual threads based on socket objects created for the sake of the identity of each clie